In [1]:
import numpy as np
import pandas as pd
import random
import os
import csv
import torch
from torchvision.models import densenet121
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn import preprocessing

In [2]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(20),      # Rotate the image by up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Randomly change brightness, contrast, and saturation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((224, 224))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((224, 224))
])


class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = torch.from_numpy(labels).type(torch.LongTensor)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
label_encoder = preprocessing.LabelEncoder()

# Load pre-trained models
densenet = densenet121(pretrained=True)

toy_data_path = 'toy_dataset_label.csv'
toy_data = pd.read_csv(toy_data_path, on_bad_lines='skip', delimiter='\t', quoting=csv.QUOTE_NONE)
labels_toy_data = toy_data.iloc[:, 9].values

classes = set()
for val in labels_toy_data:
    if pd.notna(val):
        classes.add(val)
num_classes = len(classes)
densenet.classifier = torch.nn.Linear(densenet.classifier.in_features, num_classes)

C:\Users\Buse\anaconda3\envs\comp411\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\Users\Buse\anaconda3\envs\comp411\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
#freezes up to this layer, non-inclusive
freeze_until_layer = 'features.norm5'

# Flag to mark when to start unfreezing layers
freeze = True

def freeze_layers(model, freeze_until_layer):
    freeze = True
    for name, module in model.named_modules():
        if name == freeze_until_layer:
            freeze = False
        for param in module.parameters():
            param.requires_grad = not freeze
    return model

densenet = freeze_layers(densenet, freeze_until_layer)
            
"""# You can print out to check which layers are frozen and which are not
for name, param in densenet.named_parameters():
    print(f"Layer: {name}, Frozen: {not param.requires_grad}")"""

Layer: features.conv0.weight, Frozen: True
Layer: features.norm0.weight, Frozen: True
Layer: features.norm0.bias, Frozen: True
Layer: features.denseblock1.denselayer1.norm1.weight, Frozen: True
Layer: features.denseblock1.denselayer1.norm1.bias, Frozen: True
Layer: features.denseblock1.denselayer1.conv1.weight, Frozen: True
Layer: features.denseblock1.denselayer1.norm2.weight, Frozen: True
Layer: features.denseblock1.denselayer1.norm2.bias, Frozen: True
Layer: features.denseblock1.denselayer1.conv2.weight, Frozen: True
Layer: features.denseblock1.denselayer2.norm1.weight, Frozen: True
Layer: features.denseblock1.denselayer2.norm1.bias, Frozen: True
Layer: features.denseblock1.denselayer2.conv1.weight, Frozen: True
Layer: features.denseblock1.denselayer2.norm2.weight, Frozen: True
Layer: features.denseblock1.denselayer2.norm2.bias, Frozen: True
Layer: features.denseblock1.denselayer2.conv2.weight, Frozen: True
Layer: features.denseblock1.denselayer3.norm1.weight, Frozen: True
Layer: fea

In [5]:
def train_model(model, train_loader, val_loader, num_epochs=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Training finished')
        # Validation step
        model.eval()
        total_val_loss = 0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for val_images, val_labels in val_loader:
                outputs = model(val_images)
                loss = criterion(outputs, val_labels)
                total_val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_correct += (predicted == val_labels).sum().item()
                total_samples += val_labels.size(0)

        avg_val_loss = total_val_loss / len(val_loader)
        val_accuracy = total_correct / total_samples

        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Return the trained model
    return model

def test_model(model, test_loader):
    criterion = torch.nn.CrossEntropyLoss()
    
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        test_loss = 0
        criterion = torch.nn.CrossEntropyLoss()  # Replace with your loss function

        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


        average_loss = test_loss / len(test_loader)
        accuracy = correct / total

        print(f'Test Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.4f}')

In [6]:
all_image_paths = []
for i in range(4345):
    img_path = 'toy_dataset/toy_dataset/{}.jpg'.format(i+1)
    all_image_paths.append(img_path)
    
all_labels = labels_toy_data[:4345]
train_paths, test_paths, train_labels, test_labels = train_test_split(all_image_paths, all_labels, test_size=0.2, random_state=42)

# Further split the training set into training and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.25, random_state=42)

train_dataset = CustomDataset(train_paths, label_encoder.fit_transform(train_labels), transform=train_transform)
val_dataset = CustomDataset(val_paths, label_encoder.fit_transform(val_labels), transform=test_transform)
test_dataset = CustomDataset(test_paths, label_encoder.fit_transform(test_labels), transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

trained_densenet = train_model(densenet, train_loader, val_loader, num_epochs=5)

KeyboardInterrupt: 

In [ ]:
test_model(trained_densenet, test_loader)